In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Start by importing the required libraries first
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
insurance = pd.read_csv("../input/insurance/insurance.csv")
insurance

In [ ]:
insurance_one_hot = pd.get_dummies(insurance)
insurance_one_hot.head()

In [ ]:
# Now lets create the X and y i.e. the dependent variables
X = insurance_one_hot.drop("charges",axis = 1)
X

In [ ]:
y = insurance_one_hot["charges"]
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
len(X), len(X_test), len(X_train), len(y), len(y_train), len(y_test)

In [ ]:
# Now to build a neural net for this 

tf.random.set_seed(42)

# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(loss = "mae",
             optimizer = "Adam",
             metrics = ["mae"])

# Fit the model
hist = model.fit(X_train, y_train, epochs = 500)


In [ ]:
# Now its time to evaluate the model
model.evaluate(X_test, y_test)

In [ ]:
y_train.median(), y_train.mean()

In [ ]:
pd.DataFrame(hist.history).plot()
plt.xlabel("loss")
plt.ylabel("epochs")

In [ ]:
# We still need to preprocess the data 
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

#Create a column transformer
ct = make_column_transformer(
    (MinMaxScaler(), ["age", "bmi", "children"]),
    (OneHotEncoder(handle_unknown="ignore"), ["sex", "smoker", "region"]))
    
# Create the X and y
X = insurance.drop("charges", axis = 1)
y = insurance["charges"]
    
# Build our train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
    
# Fit the column transformer to our training data
ct.fit(X_train)
    
# Transform training and test data with normalization and OneHotEncoder
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [ ]:
X_train.loc[0]

In [ ]:
X_train.shape, X_train_normal.shape

In [ ]:
# Build a neural net again to fit on our normalized data
tf.random.set_seed(42)

# Create the model
model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1)
])

# Compile the model
model2.compile(loss = "mae",
             optimizer = tf.keras.optimizers.Adam(lr = 0.01),
             metrics = ["mae"])

# Fit the model
hist = model2.fit(X_train_normal, y_train, epochs = 200)


In [ ]:
#Evaluate the model now
model2.evaluate(X_test_normal,y_test)